In [1]:
# Import Dependencies
import pandas as pd 
import json
import requests
import numpy as np
import datetime
from datetime import timedelta


In [2]:
# Read the CSV file from AWS to Pandas Dataframe
url = "https://finalproject-3.s3.us-west-1.amazonaws.com/2017.csv"

df = pd.read_csv(url)

df.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2017-01-01,AA,1,JFK,LAX,800,831.0,31.0,25.0,856.0,1143.0,26.0,1142,1209.0,27.0,0.0,NaN,0.0,402.0,398.0,347.0,2475.0,27.0,0.0,0.0,0.0,0.0,NaN
1,2017-01-01,AA,2,LAX,JFK,900,934.0,34.0,34.0,1008.0,1757.0,12.0,1727,1809.0,42.0,0.0,NaN,0.0,327.0,335.0,289.0,2475.0,34.0,0.0,8.0,0.0,0.0,NaN
2,2017-01-01,AA,4,LAX,JFK,1130,1221.0,51.0,20.0,1241.0,2025.0,15.0,1958,2040.0,42.0,0.0,NaN,0.0,328.0,319.0,284.0,2475.0,7.0,0.0,0.0,0.0,35.0,NaN
3,2017-01-01,AA,5,DFW,HNL,1135,1252.0,77.0,19.0,1311.0,1744.0,5.0,1612,1749.0,97.0,0.0,NaN,0.0,517.0,537.0,513.0,3784.0,77.0,0.0,20.0,0.0,0.0,NaN
4,2017-01-01,AA,6,OGG,DFW,1855,1855.0,0.0,16.0,1911.0,631.0,11.0,600,642.0,42.0,0.0,NaN,0.0,425.0,467.0,440.0,3711.0,0.0,0.0,42.0,0.0,0.0,NaN


In [3]:
# Filter the data for Tucson Airport
df1 = df.loc[df['ORIGIN'] == 'TUS']



# df1.to_csv('tus.csv')
# files.download('tus.csv')

# Remove the Unnamed columns which as no values
tus_df = df1.drop(['Unnamed: 27'], axis = 1)


In [4]:
# Fill all NaN with 0
from google.colab import files
tus_df1 = tus_df.fillna(0)



In [5]:
import math

def time_conv(mil_time):
  hh_mm = []
  for time in mil_time:
    if math.isnan(time):
      hh_mm.append(time)
    elif time < 59:
      hh_mm.append(datetime.datetime.strptime(str(int(time)), '%M').strftime('%H:%M'))  
    else:
      hh_mm.append(datetime.datetime.strptime(str(int(time)), '%H%M').strftime('%H:%M'))  
  return hh_mm

In [6]:
# if dep_time = 0 the File was cancelled

tus_df['CRS_DEP_HM'] = time_conv(tus_df['CRS_DEP_TIME'])
tus_df['DEP_TIME_HM'] = time_conv(tus_df['DEP_TIME'])
tus_df['WHEELS_OFF_HM'] = time_conv(tus_df['WHEELS_OFF'])
tus_df['WHEELS_ON_HM'] = time_conv(tus_df['WHEELS_ON'])
tus_df['CRS_ARR_TIME_HM'] = time_conv(tus_df['CRS_ARR_TIME'])
tus_df['ARR_TIME_HM'] = time_conv(tus_df['ARR_TIME'])

# tus_df['CRS_ARR_HM'] = pd.to_datetime(tus_df['CRS_ARR_TIME'], format = '%H%M').dt.time

# tus_df1 = tus_df1[(tus_df1['ARR_TIME'] !=0) &  (tus_df1['ARR_TIME'] !=5)]
# tus_df1['ARR_TIME_HM'] = pd.to_datetime(tus_df1['ARR_TIME'], format = '%H%M').dt.time

# tus_df1 = tus_df1[(tus_df1['WHEELS_OFF'] !=0) & (tus_df1['WHEELS_OFF'] !=3)]
# tus_df1['WHEELS_OFF_HM'] = pd.to_datetime(tus_df1['WHEELS_OFF'], format = '%H%M').dt.time

tus_df
# tus_df1.to_csv('tus.csv')
# files.download('tus.csv')

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM
73,2017-01-01,AA,98,TUS,ORD,1322,1324.0,2.0,10.0,1334.0,1728.0,10.0,1748,1738.0,-10.0,0.0,NaN,0.0,206.0,194.0,174.0,1437.0,NaN,NaN,NaN,NaN,NaN,13:22,13:24,13:34,17:28,17:48,17:38
206,2017-01-01,AA,302,TUS,DFW,605,641.0,36.0,13.0,654.0,936.0,11.0,916,947.0,31.0,0.0,NaN,0.0,131.0,126.0,102.0,813.0,31.0,0.0,0.0,0.0,0.0,06:05,06:41,06:54,09:36,09:16,09:47
323,2017-01-01,AA,1206,TUS,DFW,1355,1350.0,-5.0,14.0,1404.0,1646.0,7.0,1707,1653.0,-14.0,0.0,NaN,0.0,132.0,123.0,102.0,813.0,NaN,NaN,NaN,NaN,NaN,13:55,13:50,14:04,16:46,17:07,16:53
851,2017-01-01,EV,4220,TUS,IAH,1125,1115.0,-10.0,14.0,1129.0,1430.0,5.0,1453,1435.0,-18.0,0.0,NaN,0.0,148.0,140.0,121.0,936.0,NaN,NaN,NaN,NaN,NaN,11:25,11:15,11:29,14:30,14:53,14:35
1227,2017-01-01,OO,5479,TUS,DEN,1050,1043.0,-7.0,34.0,1117.0,1248.0,8.0,1257,1256.0,-1.0,0.0,NaN,0.0,127.0,133.0,91.0,639.0,NaN,NaN,NaN,NaN,NaN,10:50,10:43,11:17,12:48,12:57,12:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5672833,2017-12-31,EV,3904,TUS,IAH,525,520.0,-5.0,12.0,532.0,836.0,7.0,857,843.0,-14.0,0.0,NaN,0.0,152.0,143.0,124.0,936.0,NaN,NaN,NaN,NaN,NaN,05:25,05:20,05:32,08:36,08:57,08:43
5673130,2017-12-31,EV,4359,TUS,IAH,1500,1451.0,-9.0,10.0,1501.0,1757.0,11.0,1828,1808.0,-20.0,0.0,NaN,0.0,148.0,137.0,116.0,936.0,NaN,NaN,NaN,NaN,NaN,15:00,14:51,15:01,17:57,18:28,18:08
5673149,2017-12-31,EV,4403,TUS,IAH,1208,1205.0,-3.0,15.0,1220.0,1517.0,8.0,1536,1525.0,-11.0,0.0,NaN,0.0,148.0,140.0,117.0,936.0,NaN,NaN,NaN,NaN,NaN,12:08,12:05,12:20,15:17,15:36,15:25
5673413,2017-12-31,UA,288,TUS,IAH,730,725.0,-5.0,11.0,736.0,1032.0,5.0,1054,1037.0,-17.0,0.0,NaN,0.0,144.0,132.0,116.0,936.0,NaN,NaN,NaN,NaN,NaN,07:30,07:25,07:36,10:32,10:54,10:37


In [7]:
# Add Day of the week

tus_df['FL_DATE'] = pd.to_datetime(tus_df['FL_DATE'], format = '%Y/%m/%d')
tus_df['day_of_week'] = tus_df['FL_DATE'].dt.day_name()
tus_df

# tus_df1.to_csv('tus.csv')
# files.download('tus.csv')



,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM,day_of_week
73,2017-01-01,AA,98,TUS,ORD,1322,1324.0,2.0,10.0,1334.0,1728.0,10.0,1748,1738.0,-10.0,0.0,NaN,0.0,206.0,194.0,174.0,1437.0,NaN,NaN,NaN,NaN,NaN,13:22,13:24,13:34,17:28,17:48,17:38,Sunday
206,2017-01-01,AA,302,TUS,DFW,605,641.0,36.0,13.0,654.0,936.0,11.0,916,947.0,31.0,0.0,NaN,0.0,131.0,126.0,102.0,813.0,31.0,0.0,0.0,0.0,0.0,06:05,06:41,06:54,09:36,09:16,09:47,Sunday
323,2017-01-01,AA,1206,TUS,DFW,1355,1350.0,-5.0,14.0,1404.0,1646.0,7.0,1707,1653.0,-14.0,0.0,NaN,0.0,132.0,123.0,102.0,813.0,NaN,NaN,NaN,NaN,NaN,13:55,13:50,14:04,16:46,17:07,16:53,Sunday
851,2017-01-01,EV,4220,TUS,IAH,1125,1115.0,-10.0,14.0,1129.0,1430.0,5.0,1453,1435.0,-18.0,0.0,NaN,0.0,148.0,140.0,121.0,936.0,NaN,NaN,NaN,NaN,NaN,11:25,11:15,11:29,14:30,14:53,14:35,Sunday
1227,2017-01-01,OO,5479,TUS,DEN,1050,1043.0,-7.0,34.0,1117.0,1248.0,8.0,1257,1256.0,-1.0,0.0,NaN,0.0,127.0,133.0,91.0,639.0,NaN,NaN,NaN,NaN,NaN,10:50,10:43,11:17,12:48,12:57,12:56,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5672833,2017-12-31,EV,3904,TUS,IAH,525,520.0,-5.0,12.0,532.0,836.0,7.0,857,843.0,-14.0,0.0,NaN,0.0,152.0,143.0,124.0,936.0,NaN,NaN,NaN,NaN,NaN,05:25,05:20,05:32,08:36,08:57,08:43,Sunday
5673130,2017-12-31,EV,4359,TUS,IAH,1500,1451.0,-9.0,10.0,1501.0,1757.0,11.0,1828,1808.0,-20.0,0.0,NaN,0.0,148.0,137.0,116.0,936.0,NaN,NaN,NaN,NaN,NaN,15:00,14:51,15:01,17:57,18:28,18:08,Sunday
5673149,2017-12-31,EV,4403,TUS,IAH,1208,1205.0,-3.0,15.0,1220.0,1517.0,8.0,1536,1525.0,-11.0,0.0,NaN,0.0,148.0,140.0,117.0,936.0,NaN,NaN,NaN,NaN,NaN,12:08,12:05,12:20,15:17,15:36,15:25,Sunday
5673413,2017-12-31,UA,288,TUS,IAH,730,725.0,-5.0,11.0,736.0,1032.0,5.0,1054,1037.0,-17.0,0.0,NaN,0.0,144.0,132.0,116.0,936.0,NaN,NaN,NaN,NaN,NaN,07:30,07:25,07:36,10:32,10:54,10:37,Sunday


In [8]:
# Airport Codes
url_airport = "https://finalproject-3.s3.us-west-1.amazonaws.com/airports_codes.csv"

airport = pd.read_csv(url_airport)

airport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9235 entries, 0 to 9234
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   AirportCode    9235 non-null   object
 1   CityName       9235 non-null   object
 2   WorldAreaCode  9235 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 216.6+ KB


In [9]:
merge_dest_df = pd.merge(airport, tus_df, left_on='AirportCode', right_on='DEST', how='inner')

In [10]:
merge_dest_origin_df = pd.merge(airport, merge_dest_df, left_on='AirportCode', right_on='ORIGIN', how='inner')
merge_dest_origin_df

,AirportCode_x,CityName_x,WorldAreaCode_x,AirportCode_y,CityName_y,WorldAreaCode_y,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM,day_of_week
0,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2017-01-01,DL,2066,TUS,ATL,1430,1459.0,29.0,10.0,1509.0,2011.0,17.0,2003,2028.0,25.0,0.0,NaN,0.0,213.0,209.0,182.0,1541.0,11.0,0.0,0.0,0.0,14.0,14:30,14:59,15:09,20:11,20:03,20:28,Sunday
1,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2017-01-02,DL,1127,TUS,ATL,600,637.0,37.0,10.0,647.0,1157.0,6.0,1129,1203.0,34.0,0.0,NaN,0.0,209.0,206.0,190.0,1541.0,34.0,0.0,0.0,0.0,0.0,06:00,06:37,06:47,11:57,11:29,12:03,Monday
2,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2017-01-02,DL,2066,TUS,ATL,1430,1447.0,17.0,10.0,1457.0,1943.0,9.0,2005,1952.0,-13.0,0.0,NaN,0.0,215.0,185.0,166.0,1541.0,NaN,NaN,NaN,NaN,NaN,14:30,14:47,14:57,19:43,20:05,19:52,Monday
3,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2017-01-03,DL,1127,TUS,ATL,600,559.0,-1.0,29.0,628.0,1116.0,5.0,1129,1121.0,-8.0,0.0,NaN,0.0,209.0,202.0,168.0,1541.0,NaN,NaN,NaN,NaN,NaN,06:00,05:59,06:28,11:16,11:29,11:21,Tuesday
4,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2017-01-03,DL,2066,TUS,ATL,1430,1441.0,11.0,11.0,1452.0,1939.0,9.0,2005,1948.0,-17.0,0.0,NaN,0.0,215.0,187.0,167.0,1541.0,NaN,NaN,NaN,NaN,NaN,14:30,14:41,14:52,19:39,20:05,19:48,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15401,TUS,"TUCSON, ARIZONA, USA",81,SLC,"SALT LAKE CITY, UTAH, USA",87,2017-12-29,OO,4567,TUS,SLC,1227,1217.0,-10.0,16.0,1233.0,1401.0,6.0,1429,1407.0,-22.0,0.0,NaN,0.0,122.0,110.0,88.0,601.0,NaN,NaN,NaN,NaN,NaN,12:27,12:17,12:33,14:01,14:29,14:07,Friday
15402,TUS,"TUCSON, ARIZONA, USA",81,SLC,"SALT LAKE CITY, UTAH, USA",87,2017-12-30,OO,4539,TUS,SLC,810,807.0,-3.0,14.0,821.0,954.0,10.0,1018,1004.0,-14.0,0.0,NaN,0.0,128.0,117.0,93.0,601.0,NaN,NaN,NaN,NaN,NaN,08:10,08:07,08:21,09:54,10:18,10:04,Saturday
15403,TUS,"TUCSON, ARIZONA, USA",81,SLC,"SALT LAKE CITY, UTAH, USA",87,2017-12-30,OO,4567,TUS,SLC,1227,1307.0,40.0,18.0,1325.0,1446.0,4.0,1429,1450.0,21.0,0.0,NaN,0.0,122.0,103.0,81.0,601.0,0.0,0.0,0.0,0.0,21.0,12:27,13:07,13:25,14:46,14:29,14:50,Saturday
15404,TUS,"TUCSON, ARIZONA, USA",81,SLC,"SALT LAKE CITY, UTAH, USA",87,2017-12-31,OO,4539,TUS,SLC,810,759.0,-11.0,16.0,815.0,942.0,6.0,1018,948.0,-30.0,0.0,NaN,0.0,128.0,109.0,87.0,601.0,NaN,NaN,NaN,NaN,NaN,08:10,07:59,08:15,09:42,10:18,09:48,Sunday


In [11]:
# Rename Columns
rename_dest_origin_df = merge_dest_origin_df.rename(columns = {'CityName_x': 'origin_city_name',
                             'CityName_y': 'dest_city_name'})

rename_dest_origin_df

,AirportCode_x,origin_city_name,WorldAreaCode_x,AirportCode_y,dest_city_name,WorldAreaCode_y,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM,day_of_week
0,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2017-01-01,DL,2066,TUS,ATL,1430,1459.0,29.0,10.0,1509.0,2011.0,17.0,2003,2028.0,25.0,0.0,NaN,0.0,213.0,209.0,182.0,1541.0,11.0,0.0,0.0,0.0,14.0,14:30,14:59,15:09,20:11,20:03,20:28,Sunday
1,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2017-01-02,DL,1127,TUS,ATL,600,637.0,37.0,10.0,647.0,1157.0,6.0,1129,1203.0,34.0,0.0,NaN,0.0,209.0,206.0,190.0,1541.0,34.0,0.0,0.0,0.0,0.0,06:00,06:37,06:47,11:57,11:29,12:03,Monday
2,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2017-01-02,DL,2066,TUS,ATL,1430,1447.0,17.0,10.0,1457.0,1943.0,9.0,2005,1952.0,-13.0,0.0,NaN,0.0,215.0,185.0,166.0,1541.0,NaN,NaN,NaN,NaN,NaN,14:30,14:47,14:57,19:43,20:05,19:52,Monday
3,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2017-01-03,DL,1127,TUS,ATL,600,559.0,-1.0,29.0,628.0,1116.0,5.0,1129,1121.0,-8.0,0.0,NaN,0.0,209.0,202.0,168.0,1541.0,NaN,NaN,NaN,NaN,NaN,06:00,05:59,06:28,11:16,11:29,11:21,Tuesday
4,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2017-01-03,DL,2066,TUS,ATL,1430,1441.0,11.0,11.0,1452.0,1939.0,9.0,2005,1948.0,-17.0,0.0,NaN,0.0,215.0,187.0,167.0,1541.0,NaN,NaN,NaN,NaN,NaN,14:30,14:41,14:52,19:39,20:05,19:48,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15401,TUS,"TUCSON, ARIZONA, USA",81,SLC,"SALT LAKE CITY, UTAH, USA",87,2017-12-29,OO,4567,TUS,SLC,1227,1217.0,-10.0,16.0,1233.0,1401.0,6.0,1429,1407.0,-22.0,0.0,NaN,0.0,122.0,110.0,88.0,601.0,NaN,NaN,NaN,NaN,NaN,12:27,12:17,12:33,14:01,14:29,14:07,Friday
15402,TUS,"TUCSON, ARIZONA, USA",81,SLC,"SALT LAKE CITY, UTAH, USA",87,2017-12-30,OO,4539,TUS,SLC,810,807.0,-3.0,14.0,821.0,954.0,10.0,1018,1004.0,-14.0,0.0,NaN,0.0,128.0,117.0,93.0,601.0,NaN,NaN,NaN,NaN,NaN,08:10,08:07,08:21,09:54,10:18,10:04,Saturday
15403,TUS,"TUCSON, ARIZONA, USA",81,SLC,"SALT LAKE CITY, UTAH, USA",87,2017-12-30,OO,4567,TUS,SLC,1227,1307.0,40.0,18.0,1325.0,1446.0,4.0,1429,1450.0,21.0,0.0,NaN,0.0,122.0,103.0,81.0,601.0,0.0,0.0,0.0,0.0,21.0,12:27,13:07,13:25,14:46,14:29,14:50,Saturday
15404,TUS,"TUCSON, ARIZONA, USA",81,SLC,"SALT LAKE CITY, UTAH, USA",87,2017-12-31,OO,4539,TUS,SLC,810,759.0,-11.0,16.0,815.0,942.0,6.0,1018,948.0,-30.0,0.0,NaN,0.0,128.0,109.0,87.0,601.0,NaN,NaN,NaN,NaN,NaN,08:10,07:59,08:15,09:42,10:18,09:48,Sunday


In [14]:
# Drop colums not needed.
tus_final_df = rename_dest_origin_df.drop(['AirportCode_x', 'WorldAreaCode_x', 'AirportCode_y', 'WorldAreaCode_y'], axis = 1)

DELAY = []

for delay in tus_final_df['ARR_DELAY']:
  if delay < 30:
    DELAY.append(0)
  else:
    DELAY.append(1)

tus_final_df['DELAY'] = DELAY



# Export the final DF
tus_final_df.to_csv('2017_TUS.csv', index = False)
files.download('2017_TUS.csv')

tus_final_df



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,origin_city_name,dest_city_name,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM,day_of_week,DELAY
0,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2017-01-01,DL,2066,TUS,ATL,1430,1459.0,29.0,10.0,1509.0,2011.0,17.0,2003,2028.0,25.0,0.0,NaN,0.0,213.0,209.0,182.0,1541.0,11.0,0.0,0.0,0.0,14.0,14:30,14:59,15:09,20:11,20:03,20:28,Sunday,0
1,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2017-01-02,DL,1127,TUS,ATL,600,637.0,37.0,10.0,647.0,1157.0,6.0,1129,1203.0,34.0,0.0,NaN,0.0,209.0,206.0,190.0,1541.0,34.0,0.0,0.0,0.0,0.0,06:00,06:37,06:47,11:57,11:29,12:03,Monday,1
2,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2017-01-02,DL,2066,TUS,ATL,1430,1447.0,17.0,10.0,1457.0,1943.0,9.0,2005,1952.0,-13.0,0.0,NaN,0.0,215.0,185.0,166.0,1541.0,NaN,NaN,NaN,NaN,NaN,14:30,14:47,14:57,19:43,20:05,19:52,Monday,0
3,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2017-01-03,DL,1127,TUS,ATL,600,559.0,-1.0,29.0,628.0,1116.0,5.0,1129,1121.0,-8.0,0.0,NaN,0.0,209.0,202.0,168.0,1541.0,NaN,NaN,NaN,NaN,NaN,06:00,05:59,06:28,11:16,11:29,11:21,Tuesday,0
4,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2017-01-03,DL,2066,TUS,ATL,1430,1441.0,11.0,11.0,1452.0,1939.0,9.0,2005,1948.0,-17.0,0.0,NaN,0.0,215.0,187.0,167.0,1541.0,NaN,NaN,NaN,NaN,NaN,14:30,14:41,14:52,19:39,20:05,19:48,Tuesday,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15401,"TUCSON, ARIZONA, USA","SALT LAKE CITY, UTAH, USA",2017-12-29,OO,4567,TUS,SLC,1227,1217.0,-10.0,16.0,1233.0,1401.0,6.0,1429,1407.0,-22.0,0.0,NaN,0.0,122.0,110.0,88.0,601.0,NaN,NaN,NaN,NaN,NaN,12:27,12:17,12:33,14:01,14:29,14:07,Friday,0
15402,"TUCSON, ARIZONA, USA","SALT LAKE CITY, UTAH, USA",2017-12-30,OO,4539,TUS,SLC,810,807.0,-3.0,14.0,821.0,954.0,10.0,1018,1004.0,-14.0,0.0,NaN,0.0,128.0,117.0,93.0,601.0,NaN,NaN,NaN,NaN,NaN,08:10,08:07,08:21,09:54,10:18,10:04,Saturday,0
15403,"TUCSON, ARIZONA, USA","SALT LAKE CITY, UTAH, USA",2017-12-30,OO,4567,TUS,SLC,1227,1307.0,40.0,18.0,1325.0,1446.0,4.0,1429,1450.0,21.0,0.0,NaN,0.0,122.0,103.0,81.0,601.0,0.0,0.0,0.0,0.0,21.0,12:27,13:07,13:25,14:46,14:29,14:50,Saturday,0
15404,"TUCSON, ARIZONA, USA","SALT LAKE CITY, UTAH, USA",2017-12-31,OO,4539,TUS,SLC,810,759.0,-11.0,16.0,815.0,942.0,6.0,1018,948.0,-30.0,0.0,NaN,0.0,128.0,109.0,87.0,601.0,NaN,NaN,NaN,NaN,NaN,08:10,07:59,08:15,09:42,10:18,09:48,Sunday,0
